# Add, replace, cache and delete artifacts

In [ ]:
import pytest
import shutil
import lamindb as ln

In [ ]:
ln.setup.login("testuser1")

In [ ]:
try:
    root_path = ln.UPath("s3://lamindb-ci/test-add-replace-cache")
    if root_path.exists():
        root_path.rmdir()
    ln.setup.delete("testuser1/test-add-replace-cache", force=True)
except BaseException:  # noqa: S110
    pass

In [ ]:
ln.setup.init(storage="s3://lamindb-ci/test-add-replace-cache")

## Save with auto-managed (`key=None`)

In [ ]:
AUTO_KEY_PREFIX = ln.core.storage.paths.AUTO_KEY_PREFIX
root = ln.settings.storage.root

In [ ]:
artifact = ln.Artifact("./test-files/iris.csv", description="iris.csv")

In [ ]:
artifact.save()

In [ ]:
key_path = root / f"{AUTO_KEY_PREFIX}{artifact.uid}{artifact.suffix}"
assert key_path.exists()

In [ ]:
cache_csv_path = artifact.cache()
print(cache_csv_path)
assert cache_csv_path.suffix == ".csv"

In [ ]:
artifact.replace("./test-files/iris.data")
artifact.save()

In [ ]:
old_key_path = key_path
new_key_path = root / f"{AUTO_KEY_PREFIX}{artifact.uid}{artifact.suffix}"

The suffix changed:

In [ ]:
print(old_key_path)
print(new_key_path)
assert not old_key_path.exists()
assert new_key_path.exists()

In [ ]:
cache_data_path = artifact.cache()
print(cache_data_path)
assert cache_data_path.suffix == ".data"
assert cache_data_path.stat().st_mtime >= cache_csv_path.stat().st_mtime

In [ ]:
artifact.delete(permanent=True)

## Save with manually passed real `key`

In [ ]:
ln.settings.creation._artifact_use_virtual_keys = False

In [ ]:
artifact = ln.Artifact("./test-files/iris.csv", key="iris.csv")

In [ ]:
artifact.save()

In [ ]:
key_path = root / "iris.csv"

In [ ]:
assert key_path.exists()

In [ ]:
artifact.replace("./test-files/new_iris.csv")

In [ ]:
artifact.save()

Check paths: no changes here, as the suffix didn't change.

In [ ]:
old_key_path = key_path
new_key_path = root / "new_iris.csv"

In [ ]:
old_key_path

In [ ]:
new_key_path

In [ ]:
assert old_key_path.exists()
assert not new_key_path.exists()

In [ ]:
artifact.replace("./test-files/iris.data")

In [ ]:
artifact.save()

In [ ]:
new_key_path = root / "iris.data"

In [ ]:
old_key_path

In [ ]:
new_key_path

In [ ]:
assert not old_key_path.exists()
assert new_key_path.exists()

In [ ]:
artifact.delete(permanent=True, storage=True)

## Save from memory

In [ ]:
import pandas as pd

In [ ]:
iris = pd.read_csv("./test-files/iris.csv")

In [ ]:
artifact = ln.Artifact.from_df(iris, description="iris_store", key="iris.parquet")

In [ ]:
artifact.save()

In [ ]:
key_path = root / "iris.parquet"

In [ ]:
assert key_path.exists()

In [ ]:
artifact.replace(data=iris[:-1])

In [ ]:
assert artifact.key == "iris.parquet"

In [ ]:
artifact.save()

In [ ]:
assert key_path.exists()

In [ ]:
artifact.replace("./test-files/new_iris.csv")

In [ ]:
artifact.save()

In [ ]:
old_key_path = key_path
new_key_path = root / "iris.csv"

In [ ]:
old_key_path

In [ ]:
new_key_path

In [ ]:
assert not old_key_path.exists()
assert new_key_path.exists()

In [ ]:
# we use the path in the next section
path_in_storage = artifact.path
artifact.delete(permanent=True, storage=False)

## Save with manually passed virtual `key`

In [ ]:
ln.settings.creation._artifact_use_virtual_keys = True

In [ ]:
artifact = ln.Artifact("./test-files/iris.csv", key="iris.csv")

In [ ]:
artifact.save()

In [ ]:
with pytest.raises(ValueError):
    artifact.replace(path_in_storage)

In [ ]:
# return an existing artifact if the hash is the same
assert artifact == artifact.replace("./test-files/iris.csv")

In [ ]:
fpath = artifact.path
assert fpath.suffix == ".csv" and fpath.stem == artifact.uid

In [ ]:
artifact.replace("./test-files/iris.data")

In [ ]:
artifact.save()

In [ ]:
assert artifact.key == "iris.data"

In [ ]:
assert not fpath.exists()

In [ ]:
fpath = artifact.path
assert fpath.suffix == ".data" and fpath.stem == artifact.uid

In [ ]:
artifact.delete(permanent=True, storage=True)

In [ ]:
path_in_storage.unlink()

## Replace with folder artifacts

In [ ]:
adata = ln.core.datasets.anndata_pbmc68k_reduced()

adata.write_zarr("pbmc68k.zarr")

In [ ]:
artifact = ln.Artifact("./pbmc68k.zarr", key="pbmc68k.zarr").save()
save_hash = artifact.hash
save_n_files = artifact.n_files

In [ ]:
assert save_hash is not None
assert artifact.path.is_dir()

In [ ]:
adata.obs["new_col"] = "new"

adata.write_zarr("./pbmc68k_new.zrad")

In [ ]:
artifact.replace("./pbmc68k_new.zrad")
artifact.save()

In [ ]:
assert artifact.key == "pbmc68k.zrad"
assert artifact.hash != save_hash
assert artifact.n_files != save_n_files
assert artifact.path.is_dir()

In [ ]:
shutil.rmtree("./pbmc68k.zarr")
shutil.rmtree("./pbmc68k_new.zrad")

In [ ]:
artifact.delete(permanent=True, storage=True)

In [ ]:
ln.setup.delete("test-add-replace-cache", force=True)